In [1]:
from algos.MinSel import min_selectivity
import csv
from algos.helper_functions import get_query_latency, create_file
from energy.energy_functions import *

algo_energy = []
pg_plan_energy = []

solution_energy = []
pg_exec_energy = []

solution_cost = []
pg_cost = []

solution_latency = []
pg_latency = []

algo_metadata = {}



my_tables = {"tag": 0, "site": 0, "answer": 0,
             "question": 0, "tag_question": 0, "account": 0,
             "so_user": 0, "badge": 0, "comment": 0,
             "post_link": 0}
# calculer cardinalités
conn, cursor = connect_bdd("stack")
for t in my_tables:
    query = "select count(*) from " + t
    cursor.execute(query)
    my_tables[t] = cursor.fetchall()[0][0]
# Set the path to the directory containing the files
directory_path = "STACK-queries/stack-queries"

index = 0
# Loop over all the files in the directory
for filename in os.listdir(directory_path):
    index = index + 1
    print(index)
    # Check if the file is a file and not a directory
    if os.path.isfile(os.path.join(directory_path, filename)) and index < 113:  # and filename in elements:
        # Open the file for reading
        with open(os.path.join(directory_path, filename), 'r', encoding="utf8") as file:
            # Read the contents of the file
            query = file.read()
            algo_metadata[filename] = []

            # ENERGY CODE START
            psensor = findPowerSensor("YWATTMK1-1F6860.power")
            stopDataRecording(psensor)
            clearPowerMeterCache(psensor)
            tm = time.time()
            datalog = psensor.get_dataLogger()
            datalog.set_timeUTC(time.time())
            startDataRecording(psensor)  # Power Meter starts recording power per second
            time.sleep(2.0)
            print("4 - is recording: ", psensor.get_dataLogger().get_recording())
            startTime = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S:%f')
            # ENERGY CODE END

            solution, cost = min_selectivity(query, cursor, my_tables)
            create_file("solutions/sa", filename, solution)

            endTime = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S:%f')
            # print("4-4 - is recording: ", psensor.get_dataLogger().get_recording())
            print("startTime: ", startTime, " - endTime: ", endTime)
            # YAPI.Sleep(2000)
            time.sleep(2.0)
            print("stop recording : ", datetime.now())
            stopDataRecording(psensor)
            print("7 - is recording: ", psensor.get_dataLogger().get_recording())

            (power, exec_time, energy) = getAveragePower(psensor, startTime, endTime)

            (query_power, query_exec_time, query_energy) = get_query_exec_energy(query, False)

            (query_power, query_exec_time, solution_query_energy) = get_query_exec_energy(solution, True)

            # solution_cost.append(cost)
            # algo_energy.append(energy)
            # solution_energy.append(solution_query_energy)
            # pg_energy.append(query_energy)
            # solutions.append(solution)
            algo_metadata[filename].append(energy)
            algo_metadata[filename].append(solution_query_energy)
            algo_metadata[filename].append(get_query_latency(solution, True))

        # print("solution_cost : ", solution_cost)
        # print("algo_energy :", algo_energy)
        # print("solution_energy", solution_energy)
        # print("pg_energy", pg_energy)
print("saving csv")
# open a CSV file for writing
with open('stack-solutions/sa_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # write the header row
    writer.writerow(['Filename', 'algo energy', 'Query Energy', 'Latency'])
    for filename, data in algo_metadata.items():
        writer.writerow([filename, data[0], data[1], data[2]])

----------------------------------------------------------
3b
4 - is recording:  1
query:  SELECT COUNT(*) FROM keyword AS k WHERE k.keyword LIKE '%sequel%'
query:  SELECT COUNT(*) FROM movie_info AS mi WHERE mi.info IN ('Bulgaria')
query:  SELECT COUNT(*) FROM title AS t WHERE t.production_year > 2010
File 3b.sql created in directory solutions/ms
startTime:  2023-04-12 21:14:11:935220  - endTime:  2023-04-12 21:14:30:827716
stop recording :  2023-04-12 21:14:32.829815
7 - is recording:  0
avg_power :  120.72970000000001
Time(s):  18.892496  - AVG Power(w):  120.72970000000001  - Energy(J):  2280.8853743312
4 - is recording:  1
startTime:  2023-04-12 21:14:35:678  - endTime:  2023-04-12 21:14:37:260529
stop recording :  2023-04-12 21:14:39.262713
7 - is recording:  0
avg_power :  119.979
Time(s):  1.582529  - AVG Power(w):  119.979  - Energy(J):  189.870246891
4 - is recording:  1
startTime:  2023-04-12 21:14:42:120  - endTime:  2023-04-12 21:14:42:344650
stop recording :  2023-04-12 2